In [84]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms, utils
from torchvision.transforms import Resize, ToTensor, Normalize
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler

from PIL import Image
from tqdm.auto import tqdm

from sklearn.metrics import f1_score

In [85]:
TRAIN_PATH = '/opt/ml/input/data/train/'
TRAIN_IMAGE_PATH = '/opt/ml/input/data/train/processed_train_images/'

In [86]:
train_df = pd.read_csv(os.path.join(TRAIN_PATH, 'preprocessed_train.csv'))
train_df

,Unnamed: 0,id,gender,path,mask,age,age_3,age_11,label
0,0,003792,0,/opt/ml/input/data/train/processed_train_image...,0,30,1,3,1
1,1,000036,1,/opt/ml/input/data/train/processed_train_image...,0,58,1,5,4
2,2,003880,1,/opt/ml/input/data/train/processed_train_image...,0,55,1,5,4
3,3,003136,1,/opt/ml/input/data/train/processed_train_image...,0,19,0,1,3
4,4,006167,1,/opt/ml/input/data/train/processed_train_image...,0,18,0,1,3
...,...,...,...,...,...,...,...,...,...
18888,18888,003361,1,/opt/ml/input/data/train/processed_train_image...,0,20,0,2,3
18889,18889,006170,1,/opt/ml/input/data/train/processed_train_image...,0,20,0,2,3
18890,18890,003172,1,/opt/ml/input/data/train/processed_train_image...,0,19,0,1,3
18891,18891,001531,1,/opt/ml/input/data/train/processed_train_image...,0,26,0,2,3


In [87]:
class maskDataset(Dataset):
    def __init__(self, train_df, transform, train=True):
        self.train = train
        self.transform = transform

        if self.train == True:
            self.paths = train_df['path']
            self.labels = train_df['label']
        else:
            self.paths = train_df
        

    def __getitem__(self, index):
        if self.train:
            image = Image.open(self.paths.iloc[index])
            label = self.labels.iloc[index]
        else:
            image = Image.open(self.paths[index])

        if self.transform:
            image = self.transform(image)

        if self.train:
            return image, torch.tensor(label)
        else:
            return image

    def __len__(self):
        return len(self.paths)

In [88]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2))
])

In [89]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train_df, test_size=0.2, shuffle=True, stratify=train_df['label'], random_state=34)

In [90]:
train.shape, valid.shape

((15114, 9), (3779, 9))

In [91]:
BATCH_SIZE = 128

In [92]:
train_dataset = maskDataset(train, transform)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=1)

In [93]:
valid_dataset = maskDataset(valid, transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=1)

In [94]:
next(iter(train_dataloader))[0].shape

torch.Size([128, 3, 512, 384])

In [95]:
ResNet18 = torchvision.models.resnet18(pretrained=True)

In [96]:
ResNet18.fc = torch.nn.Linear(in_features=512, out_features=18, bias=True)

torch.nn.init.kaiming_normal_(ResNet18.fc.weight)
torch.nn.init.zeros_(ResNet18.fc.bias)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ('My Device :', device)

My Device : cuda:0


In [97]:
ResNet18.to(device)

LEARNING_RATE = 1e-5
NUM_EPOCH = 5

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ResNet18.parameters(), lr=LEARNING_RATE)

dataloaders = {
    "train" : train_dataloader,
    "valid" : valid_dataloader
}

In [98]:
best_acc = 0
best_loss = 0

for epoch in tqdm(range(NUM_EPOCH)):
    for phase in ['train', 'valid']:
        running_loss = 0
        running_acc = 0

        if phase == 'train':
            ResNet18.train()
        elif phase == 'valid':
            ResNet18.eval()

        for idx, (images, labels) in enumerate(dataloaders[phase]):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                logits = ResNet18(images)
                _, pred = torch.max(logits, 1)
                loss = loss_fn(logits, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * images.size(0)
            running_acc += torch.sum(pred == labels.data)
            

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)


        print (f'[{phase}] 현재 EPOCH : {epoch}, loss : {epoch_loss}, acc : {epoch_acc}')

[train] 현재 EPOCH : 0, loss : 1.2157697855550744, acc : 0.6334524154663086
[valid] 현재 EPOCH : 0, loss : 0.49548462693354983, acc : 0.8504894971847534
[train] 현재 EPOCH : 1, loss : 0.3766939656135344, acc : 0.8957257866859436
[valid] 현재 EPOCH : 1, loss : 0.2851104377974717, acc : 0.898121178150177
[train] 현재 EPOCH : 2, loss : 0.2341741240338483, acc : 0.9320496916770935
[valid] 현재 EPOCH : 2, loss : 0.20980334603682368, acc : 0.9198200106620789
[train] 현재 EPOCH : 3, loss : 0.16553642037717126, acc : 0.9535529613494873
[valid] 현재 EPOCH : 3, loss : 0.1704786141166677, acc : 0.9301401972770691
[train] 현재 EPOCH : 4, loss : 0.12013588908237015, acc : 0.9692999720573425
[valid] 현재 EPOCH : 4, loss : 0.13433082951033168, acc : 0.9417834877967834



In [99]:
TEST_DIR = '/opt/ml/input/data/eval/'

In [100]:
# meta 데이터와 이미지 경로를 불러옵니다
submission = pd.read_csv(os.path.join(TEST_DIR, 'info.csv'))
image_dir = os.path.join(TEST_DIR, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = maskDataset(image_paths, transform, train=False)
loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
ResNet18.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm(loader):
    images = images.to(device)
    with torch.no_grad():
        logits = ResNet18(images)
        _, pred = torch.max(logits, 1)
        # pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(TEST_DIR, 'submission.csv'), index=False)
print('test inference is done!')


test inference is done!


In [101]:
sorted(submission['ans'].unique())

[0, 1, 2, 3, 4, 5]